In [ ]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import warnings
warnings.simplefilter("ignore", UserWarning)

sys.path.insert(0, '../src/prep')
from preprocess import load_data, cleanup

## Load data & preprocess

In [ ]:
file = '../src/data/LD2011_2014.txt'
data = load_data(file, hh_start=0, hh_end=-1, start='2012-01-01 00:15:00', stop="2014-01-01 00:00:00")
data = cleanup(data)

In [ ]:
data["average"] = data.mean(axis=1)

In [ ]:
data["week"] = np.array(pd.Series(data.index.to_pydatetime()).dt.isocalendar().week)
data["year"] = np.array(pd.Series(data.index.to_pydatetime()).dt.isocalendar().year)
data["day_of_week"] = np.array(pd.Series(data.index.to_pydatetime()).dt.isocalendar().day)
data["unique_week"] = data["week"] + 52 * (data["year"] - 2012)

In [ ]:
data

# Plot seasonality

In [ ]:
def plot_seasonality(times, matrix, first_week):
    xi = times   
    yi = np.linspace(1, np.shape(matrix)[1], np.shape(matrix)[1]).astype(int)
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))
    
    ax.contour(xi, yi, np.transpose(matrix), levels=range(0, 50), linewidths=0., colors='k')
                
    cntr1 = ax.contourf(xi, yi, np.transpose(matrix), levels=np.shape(matrix)[1], cmap="RdBu_r")
    fig.colorbar(cntr1, ax=ax, shrink=1, label="Electricity Consumption [kW]")
    
    ax.set_title("Consumption")
    ax.set_ylabel("Week", fontsize=10)
    ax.set_xlabel("time", fontsize=10)
    
    # xticks
    weekday = ["Mo", "Tu", "We", "Thu", "Fri", "Sa", "Su"]
    week = [datetime(year=2000, month=1, day=3, minute=0) + timedelta(minutes=i*15) for i in range(quarters_in_week)]
    custom_ticks = range(0, quarters_in_week, 48)    
    timestamps = [f"{weekday[week[t].weekday()]}\n{week[t].hour}:{week[t].minute}" for t in custom_ticks]    
    ax.set_xticklabels(timestamps)
    ax.set_xticks(custom_ticks)
    
    # y_ticks
    r = [datetime.strptime(f"2012-W{i}" + "-1", "%Y-W%W-%w").month - 1 for i in range(1, 52+1)]
    months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "okt", "nov", "dec"]
    custom_yticks = range(1, 1+np.shape(matrix)[1], 2)   
    custom_ylabels = [months[r[(first_week+w-2)%52]] for w in custom_yticks]    
    ax.set_yticklabels(custom_ylabels)
    ax.set_yticks(custom_yticks)
    
    plt.show()

In [ ]:
# number of quarters in a week
quarters_in_week = 4*24*7

# Weekly consumption matrix per specific year
matrix = []
for i in np.unique(data["unique_week"]):
    matrix += [np.array(data[data["unique_week"] == i]["average"])]

# fill zeros to the beginning and end of the weekly matrix if not complete
matrix[0] = np.append(np.zeros(quarters_in_week - len(matrix[0])), matrix[0])
matrix[-1] = np.append(matrix[-1], np.zeros(quarters_in_week - len(matrix[-1])))

# plot seasonality
times = range(quarters_in_week)
plot_seasonality(times, np.transpose(matrix), first_week = min(data_year["unique_week"]))